### 쇼핑몰의 여러 페이지에 진열된 상품 크롤링 하기, 함수 구성 해보기

## 함수를 기능별로 세분화
- 요청 및 파싱 객체 생성 : get_request_product(url), 반환
값-bs4 객체
- 전체 정보 추출 : get_page_product(bs_obj)
    - bs_obj를 전달받아서
    - get_product_info(box)를 호출(제품 1개에 대한 정보가 반환)
    - df을 생성 후 저장
- 각 제품의 정보 추출 : get_product_info(box)
    - 제품 정보 1개에 대해 제품명/가격/세일가격을 추출해서 반환

In [31]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sju02\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [32]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl
# context = ssl._create_unverified_context()
url="http://jolse.com/category/toners-mists/1019/"

##### 1.데이터 프레임 생성

In [33]:
prd_dict = {'product' :[],'price' :[],'sale_price' :[]}
df_fin = pd.DataFrame(prd_dict)

In [34]:
df_fin

,product,price,sale_price


##### 2.요청 및 파싱 객체 생성

In [35]:
def get_request_product(url) :
    url = 'http://jolse.com/category/toners-mists/1019/'
    htmls = urlopen(url)
    htmls = htmls.read()
    

    #파싱 객체 생성
    bs_obj=BeautifulSoup(htmls,'html.parser')
    return bs_obj

##### 3.페이지 내 제품 정보 추출

In [49]:
def get_product_info(box) :
    #제품 이름
    strong_tag = box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]
    #가격
    price = box.find('ul').findAll('span')[1].text
    #세일가격
    sale_price = box.find('ul').findAll('span')[-1].text

    return {'product':prd_name, 'price': price, 'sale_price':sale_price}
    print(strong_tag.text)

In [50]:
def get_page_product(url) :
    bs_obj = get_request_product(url)

    global df_fin #전역 변수 처리

    #페이지 내 전체 제품정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'}) #제품 이름, 가격, 세일가격
    for box in boxes :
        res = pd.DataFrame(get_product_info(box),index=range(1,2)) #인덱스 지정 안하면 에러나서 일시적으로 생성
        df_fin = pd.concat([df_fin,res],axis=0,ignore_index=True)

In [51]:
url

'http://jolse.com/category/toners-mists/1019/'

In [52]:
#함수 불러오기
get_page_product(url)

In [53]:
df_fin

,product,price,sale_price
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 17.85
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 16.15
5,SOME BY MI AHA BHA PHA 30 Days Miracle Toner ...,USD 24.00,USD 17.40
6,Pyunkang Yul Essence Toner 100ml,USD 14.50,USD 9.89
7,secretKey Hyaluron Aqua Soft Toner 500ml,USD 22.00,USD 18.70
8,ROUND LAB 1025 Dokdo Toner 200ml,USD 17.00,USD 14.45
9,SOME BY MI Snail Truecica Miracle Repair Tone...,USD 24.00,USD 17.40


##### 4.여러 페이지 추출
- 해당 쇼핑몰의 url 분석
- 각 페이지에 대하여 url에 파라미터가 다르게 전송됨
- https://jolse.com/category/toners-mists/1019?page= + 페이지번호
- url 작성시 페이지번호를 따로 부착

https://jolse.com/category/toners-mists/1019/?page=3

In [54]:
base_url='https://jolse.com/category/toners-mists/1019/?page='
for i in range(2,5) :
    url= base_url + str(i)
    get_page_product(url)
    print(url)

https://jolse.com/category/toners-mists/1019/?page=2
https://jolse.com/category/toners-mists/1019/?page=3
https://jolse.com/category/toners-mists/1019/?page=4


In [55]:
url=base_url+str(1) #주소가 문자라서 문자처리 필요, 인덱스라 보여지는 순서로 숫자처리 하기 위해 1 더해줌
url[-1]


'1'

In [56]:
df_fin

,product,price,sale_price
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 17.85
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 16.15
...,...,...,...
91,NEOGEN Dermalogy Real Ferment Micro Toner 150ml,USD 22.00,USD 18.70
92,Papa Recipe Tea Tree Control Skin 200ml,USD 21.00,USD 17.85
93,Papa Recipe Eggplant Clearing Peeling Pad Ton...,USD 18.00,USD 15.30
94,Papa Recipe Eggplant Clearing Peeling Pad Ton...,USD 28.00,USD 23.80


In [57]:
#수집 데이터 저장
df_fin.to_csv('./crawl_data/beauty_price.csv',index=False)

In [58]:
df_fin

,product,price,sale_price
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 17.85
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 16.15
...,...,...,...
91,NEOGEN Dermalogy Real Ferment Micro Toner 150ml,USD 22.00,USD 18.70
92,Papa Recipe Tea Tree Control Skin 200ml,USD 21.00,USD 17.85
93,Papa Recipe Eggplant Clearing Peeling Pad Ton...,USD 18.00,USD 15.30
94,Papa Recipe Eggplant Clearing Peeling Pad Ton...,USD 28.00,USD 23.80


## 연습문제
- best 상품에 대해서 첫번째 페이지에서만 추출되고 그 후 페이지에서는 추출되지 않도록 **함수 수정**
- 전체 페이지가 몇페이지인지를 크롤링을 통해서 얻어오고 해당 페이지 수 만큼 크롤링이 되게 코드 수정(위에랑 이어서)


In [59]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl
# context = ssl._create_unverified_context()
# url="http://jolse.com/category/toners-mists/1019/" #페이지번호 있는 링크를 밑에서 작업

In [60]:
prd_dict = {'product' :[],'price' :[],'sale_price' :[]}
df_fin = pd.DataFrame(prd_dict)

In [61]:
#요청 및 파싱 객체 생성
def get_request_product(url):
    htmls = urlopen(url)
    htmls = htmls.read()

#2.파싱객체 생성
    bs_obj = BeautifulSoup(htmls, 'html.parser')
    return bs_obj

In [74]:
#한개의 제품에 대한 정보 추출
def get_product_info(box) :
    strong_tag=box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]

#가격
    price = box.find('ul').findAll('span')[1].text
    sale_price=box.find('ul').findAll('span')[-1].text
    return {'product':prd_name, 'price':price, 'sale_price':sale_price}

In [83]:
#초기 호출 함수
def get_page_product(url):
    global df_fin
    #요청 및 파싱 객체 생성
    bs_obj= get_request_product(url)

    #페이지 내 전체 제품정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})

    #1페이지가 아니면 추천제품은 다시 크롤링 하지 않게 조건식 생성
    if url.split('=')[1] !='1' :
        boxes =boxes[5:len(boxes)]

    #각 제품에 대한 정보 추출
    for box in boxes :
        res = pd.DataFrame(get_product_info(box), index=range(1,2))
        df_fin = pd.concat([df_fin,res], axis=0, ignore_index=True)


In [85]:
base_url='https://jolse.com/category/toners-mists/1019/?page='
for i in range(1,5):
    url = base_url+str(i)
    get_page_product(url)

In [86]:
df_fin

,product,price,sale_price
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 17.85
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 16.15
...,...,...,...
76,NEOGEN Dermalogy Real Ferment Micro Toner 150ml,USD 22.00,USD 18.70
77,Papa Recipe Tea Tree Control Skin 200ml,USD 21.00,USD 17.85
78,Papa Recipe Eggplant Clearing Peeling Pad Ton...,USD 18.00,USD 15.30
79,Papa Recipe Eggplant Clearing Peeling Pad Ton...,USD 28.00,USD 23.80


In [87]:
df_fin[df_fin['product']==df_fin['product'][0]]

,product,price,sale_price
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 17.85


In [96]:
url.split('=')[1]

'4'

In [99]:
#마지막 페이지 추출
import ssl
context = ssl._create_unverified_context() #파이썬에서 웹페이지에 https로 접속->인증 없이
url='https://jolse.com/category/toners-mists/1019/'
html = urlopen(url, context=context)
htmls = html.read()

In [109]:
#2.파싱객체 생성
bs_obj=BeautifulSoup(htmls, 'html.parser')
last_page=bs_obj.find('a',{'class':'last'})
last_page 
#<a class="last" href="?page=30"><img alt="Last" src="//img.echosting.cafe24.com/skin/base/common/btn_page_last.gif"/></a>

last_page=bs_obj.find('a',{'class':'last'})['href']
last_page #'?page=30'

last_page=bs_obj.find('a',{'class':'last'})['href'].split('=')[1]
last_page #30
int(last_page)
 

30

In [110]:
base_url='https://jolse.com/category/toners-mists/1019/?page='
for i in range(1,int(last_page)+1):
    url = base_url+str(i)
    get_page_product(url)

In [111]:
df_fin

,product,price,sale_price
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 17.85
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 16.15
...,...,...,...
636,MISSHA MISA Geum Sul Toner 145ml,USD 29.16,USD 18.22
637,Wish Formula Day Day 365 All In One Boosting ...,USD 25.11,USD 21.34
638,Sulwhasoo Snowise Brightening Water 125ml,USD 57.98,USD 49.28
639,TONYMOLY Tony Lab AC Control Toner 180ml,USD 18.07,USD 15.36
